In [1]:
from Robinhood import Robinhood
from Portfolio import Portfolio
from Portfolio import SIconverter
import numpy as np

In [4]:
w = SIconverter(
    buffer_size = 200,
    trader = trader,
    load = None
)

In [17]:
s = w.query_S2I("D")

In [21]:
"asd".find("https:")

-1

In [15]:
w.freq

{'BAC': 0.2,
 'D': 1.0,
 'MMM': 0.5,
 'X': 0.3333333333333333,
 'YRD': 0.3333333333333333}

In [6]:
class PortfolioMgr:
    def __init__(
        self,
        robin_un = None,
        robin_pd = None,
        name = None,
        load_from = None
    ):
        assert robin_un is not None
        assert robin_pd is not None
        assert name is not None
        self.trader = Robinhood()
        self.trader.login(robin_un,robin_pd)
        self.name = name
        self.converter = SIconverter(trader = self.trader)
        self.unassigned_bp = float(self.trader.get_account()['margin_balances']['unallocated_margin_cash'])
        self.unassigned_shares = {
            self.converter(d['instrument']):int(float(d['quantity'])) for d in self.trader.securities_owned()['results']
        }
        self.portfolios = {}
        
        
    def add_portfolio(
        self,
        name = None,
        ini_bp = 0,
        load = None,
        cancel_count = np.inf
    ):
        assert name is not None
        assert self.check_valid() == 'valid'
        assert ini_bp < self.unassigned_bp
        self.portfolios[name] = Portfolio(
            trader = self.trader,
            name = name,
            iniFund = ini_bp,
            cancel_count = np.inf
        )
        self.unassigned_bp -= ini_bp
        
        
    def update_allocatable_buying_power(self):
        allocated = 0
        for k,p in self.portfolios.items():
            allocated += p.bp
        self.unassigned_bp = self.get_bp_owned() - allocated
    
    def update_allocatable_shares(self):
        owned_shares = self.get_securities_owned()
        for k,p in self.portfolios.items():
            p.portfolio_record_lock.acquire()
            for scode in p.portfolio_record.index:
                owned_shares[scode] -= p.portfolio_record.loc[scode]["SHARES"]
            p.portfolio_record_lock.release()
        self.unassigned_shares = owned_shares
    
    def get_securities_owned(self):
        return {
            self.converter(d['instrument']):int(d['quantity']) for d in self.trader.securities_owned()['results']
        }
    
    def get_bp_owned(self):
        return float(self.trader.get_account()['margin_balances']['unallocated_margin_cash'])
    
    def add_bp_to(self,name,amount):
        assert name in self.portfolios
        self.update_allocatable_buying_power()
        assert self.unassigned_bp > amount
        self.portfolios[name].bp += amount
        self.unassigned_bp -= amount
        self.portfolios[name].add_trading_record("None","None",amount,1,"add bp")
    
    def add_shares_to(self,name,scode,amount):
        assert name in self.portfolios
        self.update_allocatable_shares()
        amount = int(amount)
        assert self.unassigned_shares[scode] > amount
        self.portfolios[name].add_shares_from_pool(scode = scode,n = amount)
        self.unassigned_shares[scode] -= amount
        self.portfolios[name].add_trading_record("None",scode,"None",amount,"add share")
        
    
    def draw_bp_from(self,name,amount):
        assert name in self.portfolios
        assert self.portfolios[name].bp > amount
        self.portfolios[name].bp -= amount
        self.unassigned_bp += amount
        self.portfolios[name].add_trading_record("None","None",amount,1,"draw bp")
    
    def draw_shares_from(self,name,scode,amount):
        assert name in self.portfolios
        amount = int(amount)
        assert self.portfolios[scode].shares_owned(scode) > amount
        self.portfolios[scode].portfolio_record_lock.acquire()
        self.portfolios[scode].portfolio_record.loc[scode]["SHARES"] -= amount
        self.portfolios[scode].portfolio_record_lock.release()
        self.unassigned_shares[scode] += amount
        self.portfolios[scode].add_trading_record("None",scode,"None",amount,"draw share")
        
    
    def transfer_bp(self,from_name,to_name,amount):
        assert from_name in self.portfolios
        assert to_name in self.portfolios
        self.portfolios[from_name].transfer_buying_power(self.portfolios[to_name],amount)
        
        
        
    def transfer_shares(self,from_name,to_name,scode,amount):
        assert from_name in self.portfolios
        assert to_name in self.portfolios
        self.portfolios[from_name].transfer_shares(self.portfolios[to_name],scode,amount)
        
        
    def check_valid(self):
        allocated_bp = 0
        allocated_shares = {}
        for k,p in self.portfolios.items():
            allocated_bp += p.bp
            p.portfolio_record_lock.acquire()
            for scode in p.portfolio_record.index:
                try:
                    allocated_shares[index] += p.portfolio_record.loc[scode]['SHARES']
                except:
                    allocated_shares[index] = p.portfolio_record.loc[scode]['SHARES']
            p.portfolio_record_lock.release()
        if allocated_bp + self.unassigned_bp > self.get_bp_owned():
            return "bp invalid"
        total_shares = self.get_securities_owned()
        for scode in total_shares.keys():
            if scode not in allocated_shares:
                alls = 0
            else:
                alls = allocated_shares[scode]
            if scode not in self.unassigned_shares:
                unas = 0
            else:
                unas = self.unassigned_shares[scode]
            if alls + unas > total_shares[scode]:
                return "scode {} invalid".format(scode)
        return "valid"
            
            
            
            
            
        
        

In [7]:
a = PortfolioMgr('hang2','zhhISbest2',"Default")

In [56]:
int('27.00')

ValueError: invalid literal for int() with base 10: '27.00'

In [2]:
trader = Robinhood()

In [52]:
trader.login('hang2','zhhISbest2')

True

In [9]:
t2 = Robinhood()

In [10]:
t2.login('hang2','zhhISbest2')

True

In [41]:
t3 = Robinhood()

In [51]:
trader.logout()

<Response [200]>

In [15]:
trader.session.get('https://api.robinhood.com/instruments/78bde958-d99c-40a8-90c5-904fd466977a/').json()['symbol']

'YRD'

In [28]:
d = {1:4,3:0}

In [32]:
max(d,key = d.get)

1

In [20]:
(trader.instruments('YRD')[0])

{'bloomberg_unique': 'EQ0000000046314223',
 'country': 'CN',
 'day_trade_ratio': '0.2500',
 'fundamentals': 'https://api.robinhood.com/fundamentals/YRD/',
 'id': '78bde958-d99c-40a8-90c5-904fd466977a',
 'list_date': '2015-12-18',
 'maintenance_ratio': '0.4500',
 'margin_initial_ratio': '0.5600',
 'market': 'https://api.robinhood.com/markets/XNYS/',
 'min_tick_size': None,
 'name': 'Yirendai Ltd.',
 'quote': 'https://api.robinhood.com/quotes/YRD/',
 'simple_name': 'Yirendai',
 'splits': 'https://api.robinhood.com/instruments/78bde958-d99c-40a8-90c5-904fd466977a/splits/',
 'state': 'active',
 'symbol': 'YRD',
 'tradability': 'tradable',
 'tradeable': True,
 'type': 'adr',
 'url': 'https://api.robinhood.com/instruments/78bde958-d99c-40a8-90c5-904fd466977a/'}

In [23]:
assert False

AssertionError: 